Study the relationship between the velocity width of HI detections and the distance to the nearest neighbor for MaNGA galaxies.

In [7]:
from astropy.table import Table

import numpy as np

# Data

In [3]:
data_directory1 = '../RotationCurves/data/HI/'
data_directory2 = '../RotationCurves/spirals/'
data_directory3 = '/Users/kellydouglass/Documents/Drexel/Research/Data/'

mangaHI_filename = data_directory1 + 'mangaHIall.fits'
alfaHI_filename = data_directory1 + 'manga_dr15_alfalfa.fits'
manga_filename = data_directory2 + 'Pipe3D-master_file_vflag_10_smooth2p27_N2O2_noWords.txt'
#kias7_filename = data_directory3 + 'kias1033_5_P-MJD-F_MPAJHU_ZdustOS_stellarMass_BPT_SFR_NSA_vflag.txt'
kias7_filename = data_directory3 + 'kias1033_5_P-MJD-F_MPAJHU_ZdustOS_stellarMass_BPT_SFR_NSA_correctvflag.txt'

mangaHI = Table.read(mangaHI_filename, format='fits')
alfaHI = Table.read(alfaHI_filename, format='fits')
manga = Table.read(manga_filename, format='ascii.commented_header')
kias7 = Table.read(kias7_filename, format='ascii.commented_header')

## Add HI data to MaNGA table

##### Initialize new columns in MaNGA data table

In [8]:
manga['WF50'] = np.NaN*np.ones(len(manga), dtype=float)
manga['WP20'] = np.NaN*np.ones(len(manga), dtype=float)
manga['logHI'] = np.NaN*np.ones(len(manga), dtype=float)

##### Add ALFALFA HI data

In [9]:
for i in range(len(alfaHI)):
    
    galID = alfaHI['PLATEIFU'][i]
    plate, IFU = galID.split('-')
    
    # Find the corresponding row index for this galaxy in the MaNGA data table
    plate_boolean = manga['MaNGA_plate'] == plate
    IFU_boolean = manga['MaNGA_IFU'] == IFU
    manga_boolean = np.logical_and(plate_boolean, IFU_boolean)
    
    # Add data to MaNGA data table
    manga['WF50'][manga_boolean] = alfaHI['WF50'][i]
    manga['WP20'][manga_boolean] = alfaHI['WP20'][i]
    manga['logHI'][manga_boolean] = alfaHI['LOGMHI'][i]

/Users/kellydouglass/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1020: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)


##### Add GBT HI data

In [11]:
for i in range(len(mangaHI)):
    
    galID = mangaHI['plateifu'][i]
    plate, IFU = galID.split('-')
    
    # Find the corresponding row index for this galaxy in the MaNGA data table
    plate_boolean = manga['MaNGA_plate'] == plate
    IFU_boolean = manga['MaNGA_IFU'] == IFU
    manga_boolean = np.logical_and(plate_boolean, IFU_boolean)
    
    # Add data to MaNGA data table
    manga['WF50'][manga_boolean] = mangaHI['WF50'][i]
    manga['WP20'][manga_boolean] = mangaHI['WP20'][i]
    manga['logHI'][manga_boolean] = mangaHI['logMHI'][i]

In [ ]:
# This cell is no longer necessary, since we put all the HI data into the main MaNGA data table
'''
NSA_plate_manga = []
NSA_fiberID_manga = []
NSA_MJD_manga = []

for i in range(len(mangaHI)):
    for j in range(len(table2)):
        if mangaHI['Plate'][i] == table2['MaNGA_plate'][j] and mangaHI['IFU'][i] == table2['MaNGA_IFU'][j]:
            NSA_plate_manga.append(table2['NSA_plate'][j])
            NSA_fiberID_manga.append(table2['NSA_fiberID'][j])
            NSA_MJD_manga.append(table2['NSA_MJD'][j])

#mangaHI.remove_column('plate')
#mangaHI.remove_column('fiberID')
#mangaHI.remove_column('MJD')
mangaHI.add_column(Column(np.asarray(NSA_plate_manga, dtype=np.float64, order='F')), index = 2, name = 'plate')
mangaHI.add_column(Column(np.asarray(NSA_fiberID_manga, dtype=np.float64, order='F')), index = 3, name = 'fiberID')
mangaHI.add_column(Column(np.asarray(NSA_MJD_manga, dtype=np.float64, order='F')), index = 4, name = 'MJD')
''';

In [ ]:
# This cell is no longer necessary, since we put all the HI data into the main MaNGA data table
'''
NSA_plate_alfa = []
NSA_fiberID_alfa = []
NSA_MJD_alfa = []

for i in range(len(alfaHI)):
    for j in range(len(table2)):
        if alfaHI['Plate'][i] == table2['MaNGA_plate'][j] and alfaHI['IFU'][i] == table2['MaNGA_IFU'][j]:
            NSA_plate_alfa.append(table2['NSA_plate'][j])
            NSA_fiberID_alfa.append(table2['NSA_fiberID'][j])
            NSA_MJD_alfa.append(table2['NSA_MJD'][j])

#alfaHI.remove_column('plate')
#alfaHI.remove_column('fiberID')
#alfaHI.remove_column('MJD')
alfaHI.add_column(Column(np.asarray(NSA_plate_alfa, dtype=np.float64, order='F')), index = 2, name = 'plate')
alfaHI.add_column(Column(np.asarray(NSA_fiberID_alfa, dtype=np.float64, order='F')), index = 3, name = 'fiberID')
alfaHI.add_column(Column(np.asarray(NSA_MJD_alfa, dtype=np.float64, order='F')), index = 4, name = 'MJD')
'''

## Isolate those MaNGA galaxies with HI detections

We only want to analyze those galaxies in MaNGA with HI detections, so let's create a subset of the main MaNGA data table that contains only those galaxies with HI detections.

In [12]:
HI_boolean = np.logical_or.reduce((np.isfinite(manga['WF50']), 
                                   np.isfinite(manga['WP20']), 
                                   np.isfinite(manga['logHI'])))

manga_HI = manga[HI_boolean]

# Find distance to nearest neighbor for each MaNGA galaxy

For each galaxy in the MaNGA data table, find its nearest neighbor and calculate the distance to that galaxy.
1. Calculate the distance to every galaxy in SDSS DR7 for each MaNGA galaxy with an HI detection.
2. Find the galaxy with the smallest distance - this is our nearest neighbor!  (Note: We need to check to make sure that we do not identify the MaNGA HI target galaxy as the nearest neighbor.)

### Convert (ra, dec, redshift) to (x, y, z)

Since we are going to do this for both the KIAS galaxies and the MaNGA galaxies, let's write a function.

In [20]:
c = 3e5 #3*(10**8) m/s <-- Scientific notation
H = 100 #2.43 * (10**-18) 1/s <-- We are doing to do all of our distances in Mpc/h (where H0 = 100h).

def rdz_to_xyz(data):
    '''
    Convert ra, dec, redshift to x, y, z
    
    
    PARAMETERS
    ==========
    
    data : astropy table
        Galaxies to analyze.  Columns include ra, dec, redshift.
        
        
    RETURNS
    =======
    
    data : astropy table
        Same as input data object, with the additional fields of x, y, z
    '''

    '''
    # We can do this all at once, since we are doing the same calculation on every 
    # row of the data table.
    
    for i in range(len(data)):
        radius_2 = c*data['redshift']/H
        r_2 = radius_2
        x_2_cor = r_2 * np.cos(kias7['ra'][i]* np.pi/180) * np.cos(kias7['dec'][i]* np.pi/180)
        y_2_cor = r_2 * np.sin(kias7['ra'][i]* np.pi/180) * np.cos(kias7['dec'][i]* np.pi/180)
        z_2_cor = r_2 * np.sin(kias7['ra'][i]* np.pi/180)
        x_2.append(x_2_cor)
        y_2.append(y_2_cor)
        z_2.append(z_2_cor)
    '''
    
    ################################################################################
    # Our data tables have different column names, so we need to determine the 
    # correct names.
    #-------------------------------------------------------------------------------
    if 'redshift' in data.colnames:
        redshift = data['redshift']
        ra = data['ra'] * np.pi/180
        dec = data['dec'] * np.pi/180
    elif 'NSA_redshift' in data.colnames:
        redshift = data['NSA_redshift']
        ra = data['NSA_RA'] * np.pi/180
        dec = data['NSA_DEC'] * np.pi/180
    else:
        print('Unknown field names.  Add to rdz_to_xyz function and rerun.')
        return data
    ################################################################################
    
    
    ################################################################################
    # Convert from spherical to cartesian coordinates
    #-------------------------------------------------------------------------------
    r = c*redshift/H
    
    data['x'] = r * np.cos(ra) * np.cos(dec)
    data['y'] = r * np.sin(ra) * np.cos(dec)
    data['z'] = r * np.sin(dec)
    ################################################################################
    
    return data

In [21]:
# def min_distance(table1, table2, ra_1, dec_1, redshift_1, ra_2, dec_2, redshift_2):
   
#     c = 3*(10**8) #m/s
#     H = 2.43 * (10**-18) #1/s
   
#     distances = []
   
#     for i in range(len(table1)):
        
#     r_1 = c*redshift_1/H
#     x_1 = r_1 * np.cos(ra_1* np.pi/180) * np.cos(dec_1 * np.pi/180)
#     y_1 = r_1 * np.sin(ra_1* np.pi/180) * np.cos(dec_1 * np.pi/180)
#     z_1 = r_1 * np.sin(ra_1* np.pi/180)
    
#     r_2 = c*redshift_2/H
#     x_2 = r_2 * np.cos(ra_2* np.pi/180) * np.cos(dec_2 * np.pi/180)
#     y_2 = r_2 * np.sin(ra_2* np.pi/180) * np.cos(dec_2 * np.pi/180)
#     z_2 = r_2 * np.sin(ra_2* np.pi/180)
    
#     distance = np.sqrt((x_1-x_2)**2) + ((y_1-y_2)**2) + ((z_1-z_2)**2))
    
#     distance_Mpc = (distance) * (3.24 * 10**-17) * (10**-6)

'''
min_distance = []  
x_1 = []
y_1 = []
z_1 = []
x_2 = []
y_2 = []
z_2 = []

   
for i in range(len(kias7)):
    radius_2 = c*kias7['redshift'][i]/H
    r_2 = radius_2
    x_2_cor = r_2 * np.cos(kias7['ra'][i]* np.pi/180) * np.cos(kias7['dec'][i]* np.pi/180)
    y_2_cor = r_2 * np.sin(kias7['ra'][i]* np.pi/180) * np.cos(kias7['dec'][i]* np.pi/180)
    z_2_cor = r_2 * np.sin(kias7['ra'][i]* np.pi/180)
    x_2.append(x_2_cor)
    y_2.append(y_2_cor)
    z_2.append(z_2_cor)
    
for j in range(len(mangaHI)):
    radius_1 = c*mangaHI['NSA_redshift'][j]/H
    r_1 = radius_1
    x_1_cor = r_1 * np.cos(mangaHI['objra'][j]* np.pi/180) * np.cos(mangaHI['objdec'][j]* np.pi/180)
    y_1_cor = r_1 * np.sin(mangaHI['objra'][j]* np.pi/180) * np.cos(mangaHI['objdec'][j]* np.pi/180)
    z_1_cor = r_1 * np.sin(mangaHI['objra'][j]* np.pi/180)
    x_1.append(x_1_cor)
    y_1.append(y_1_cor)
    z_1.append(z_1_cor)
'''

# Now that we have our function written, we can do this conversion on both data tables
manga_HI = rdz_to_xyz(manga_HI)
kias7 = rdz_to_xyz(kias7)

### Calculate distance between galaxies and find the closest one

In [22]:
# Add a column to the MaNGA data table to store our nearest-neighbor distance
manga_HI['d_nn'] = np.zeros(len(manga_HI), dtype=float) # Feel free to change this column name to whatever you want.

#for k in range(len(mangaHI)):
for k in range(len(manga_HI)):
    
    minimum = 1000 # We want to reset the minimum outside this inner for-loop, since we want to reset it only once for each mangaHI galaxy
    
    for l in range(len(kias7)):
        #minimum = 1000
        #distance = np.sqrt((x_1[k] - x_2[k])**2 + (y_1[k]-y_2[l])**2 + (z_1[k]-z_2[l])**2) * (3.24 * 10**-17) * (10**-6)
        # Careful with your indices!  You were using k to index into both the mangaHI coordinates and the kias7 coordinates
        distance = np.sqrt((manga_HI['x'][k] - kias7['x'][l])**2 + \
                           (manga_HI['y'][k] - kias7['y'][l])**2 + \
                           (manga_HI['z'][k] - kias7['z'][l])**2)
        
        #if (mangaHI['plate'][k], mangaHI['fiberID'][k], mangaHI['MJD'][k]) != (kias7['plate'][l], kias7['fiberID'][l], kias7['MJD'][l]):
        if (manga_HI['NSA_plate'][k], manga_HI['NSA_fiberID'][k], manga_HI['NSA_MJD'][k]) != \
           (kias7['plate'][l], kias7['fiberID'][l], kias7['MJD'][l]):
            if distance < minimum:
                minimum = distance
    
    #min_distance.append(minimum)
    manga_HI['d_nn'][k] = minimum